In [2]:
!cp /home/ec2-user/SageMaker/WipCoe/Notebooks_LatestCode/WiproFormat/train.csv /home/ec2-user/SageMaker/WipCoe/titanic/origdata.csv

In [26]:
df_scr=pd.read_csv("origdata.csv")
df_scr.drop(columns=['Survived'],inplace=True)
##add score file name as per lambda func call-
df_scr['scorefilename']=("scrfromnotebook.csv")
df_scr.to_csv("scorewolabel.csv",index=False)

In [37]:
!aws s3 cp scorewolabel.csv s3://ds-mlops-dev/titanic/data/scoreinput/

upload: ./scorewolabel.csv to s3://ds-mlops-dev/titanic/data/scoreinput/scorewolabel.csv


# Push data to trigger scoring

In [41]:
!aws s3 cp scorewolabel.csv s3://wi-cred-datalake-dev-raw/titanic/data/scoreinput/

upload: ./scorewolabel.csv to s3://wi-cred-datalake-dev-raw/titanic/data/scoreinput/scorewolabel.csv


In [33]:
#%%writefile titanic-score-preprocessing-ll-script.py
# Importing required library
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

# you can put any value here according to your situation
chunksize = 10000
from sklearn import preprocessing
import glob
import time
import uuid
import datetime
runtime=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
unqstr=datetime.datetime.now().strftime("%Y%m%d%H%M%S")
#path = r'/opt/ml/processing/input' # Input path
#all_files = glob.glob(path + "/*.csv")
#read them into pandas
all_files=["scorewolabel.csv"]
#filname=all_files[0].split('/')[-1]
#filname=filname.split('.')[0]
df_list = [pd.read_csv(filename,nrows=100000) for filename in all_files]
data = pd.concat(df_list)
df = data.copy()
df_list = [pd.read_csv(filename) for filename in all_files]
data = pd.concat(df_list)
df = data.copy()
print(df.head(5))
#change this as per the preprocesing used for training
#create all categorical variables that we did above for both training and test sets 
df['cabin_multiple'] = df.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
df['cabin_adv'] = df.Cabin.apply(lambda x: str(x)[0])
df['numeric_ticket'] = df.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
df['ticket_letters'] = df.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
df['name_title'] = df.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
#impute nulls for continuous data 
#df.Age = df.Age.fillna(training.Age.mean())
df.Age = df.Age.fillna(df.Age.median())
#df.Fare = df.Fare.fillna(training.Fare.mean())
df.Fare = df.Fare.fillna(df.Fare.median())
#drop null 'embarked' rows. Only 2 instances of this in training and 0 in test 
df.dropna(subset=['Embarked'],inplace = True)
#tried log norm of sibsp (not used)
df['norm_sibsp'] = np.log(df.SibSp+1)
# log norm of fare (used)
df['norm_fare'] = np.log(df.Fare+1)
# converted fare to category for pd.get_dummies()
df.Pclass = df.Pclass.astype(str)
#created dummy variables from categories (also can use OneHotEncoder)
all_dummies = pd.get_dummies(df[['PassengerId','Pclass','Sex','Age','SibSp','Parch','norm_fare','Embarked','cabin_adv','cabin_multiple','numeric_ticket','name_title']])
#scaling
encoded_df = all_dummies.copy()
encoded_df['scorefilename']=df['scorefilename']
print(encoded_df.head(5))
encoded_df[['Age','SibSp','Parch','norm_fare']]= scale.fit_transform(encoded_df[['Age','SibSp','Parch','norm_fare']])
encoded_df['runtime']=runtime
encoded_df['modelname']='Linear Learner'
encoded_df['infertype']='Batch'
encoded_df['PassengerId']=df.PassengerId.apply(lambda x:str(x)+unqstr)
encoded_df.insert(0,'runtime',encoded_df.pop('runtime'))
encoded_df.insert(1,'modelname',encoded_df.pop('modelname'))
encoded_df.insert(2,'infertype',encoded_df.pop('infertype'))
encoded_df.insert(3,'PassengerId',encoded_df.pop('PassengerId'))
encoded_df.insert(4,'scorefilename',encoded_df.pop('scorefilename'))
encoded_df
#encoded_df.to_csv("score_prooutput.csv",index=False,header=False)
#encoded_df.to_csv("/opt/ml/processing/output/test/"+filname+"_{}.csv".format(uuid.uuid1().time_low), index=False, header=False) # test data 

   PassengerId  Pclass                                               Name  \
0            1       3                            Braund, Mr. Owen Harris   
1            2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3       3                             Heikkinen, Miss. Laina   
3            4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4            5       3                           Allen, Mr. William Henry   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  \
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S   
1  female  38.0      1      0          PC 17599  71.2833   C85        C   
2  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S   
3  female  35.0      1      0            113803  53.1000  C123        S   
4    male  35.0      0      0            373450   8.0500   NaN        S   

         scorefilename  
0  scrfromnotebook.csv  
1  scrfromnotebook.csv  
2  scrfromn

,runtime,modelname,infertype,PassengerId,scorefilename,Age,SibSp,Parch,norm_fare,cabin_multiple,...,name_title_Master,name_title_Miss,name_title_Mlle,name_title_Mme,name_title_Mr,name_title_Mrs,name_title_Ms,name_title_Rev,name_title_Sir,name_title_the Countess
0,2022-08-08 07:28:51,Linear Learner,Batch,120220808072851,scrfromnotebook.csv,-0.563674,0.431350,-0.474326,-0.877591,0,...,0,0,0,0,1,0,0,0,0,0
1,2022-08-08 07:28:51,Linear Learner,Batch,220220808072851,scrfromnotebook.csv,0.669217,0.431350,-0.474326,1.366380,1,...,0,0,0,0,0,1,0,0,0,0
2,2022-08-08 07:28:51,Linear Learner,Batch,320220808072851,scrfromnotebook.csv,-0.255451,-0.475199,-0.474326,-0.796281,0,...,0,1,0,0,0,0,0,0,0,0
3,2022-08-08 07:28:51,Linear Learner,Batch,420220808072851,scrfromnotebook.csv,0.438050,0.431350,-0.474326,1.066796,1,...,0,0,0,0,0,1,0,0,0,0
4,2022-08-08 07:28:51,Linear Learner,Batch,520220808072851,scrfromnotebook.csv,0.438050,-0.475199,-0.474326,-0.781901,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2022-08-08 07:28:51,Linear Learner,Batch,88720220808072851,scrfromnotebook.csv,-0.178396,-0.475199,-0.474326,-0.330815,0,...,0,0,0,0,0,0,0,1,0,0
887,2022-08-08 07:28:51,Linear Learner,Batch,88820220808072851,scrfromnotebook.csv,-0.794841,-0.475199,-0.474326,0.491068,1,...,0,1,0,0,0,0,0,0,0,0
888,2022-08-08 07:28:51,Linear Learner,Batch,88920220808072851,scrfromnotebook.csv,-0.101340,0.431350,2.006119,0.245663,0,...,0,1,0,0,0,0,0,0,0,0
889,2022-08-08 07:28:51,Linear Learner,Batch,89020220808072851,scrfromnotebook.csv,-0.255451,-0.475199,-0.474326,0.491068,1,...,0,0,0,0,1,0,0,0,0,0


In [20]:
%%writefile titanic-score-preprocessing-xgb-script.py
# Importing required library
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
# you can put any value here according to your situation
chunksize = 10000
from sklearn import preprocessing
import glob
import time
import uuid
import datetime
runtime=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
unqstr=datetime.datetime.now().strftime("%Y%m%d%H%M%S")
path = r'/opt/ml/processing/input' # Input path
all_files = glob.glob(path + "/*.csv")
#read them into pandas
filname=all_files[0].split('/')[-1]
filname=filname.split('.')[0]
df_list = [pd.read_csv(filename,nrows=100000) for filename in all_files]
data = pd.concat(df_list)
df = data.copy()
df_list = [pd.read_csv(filename) for filename in all_files]
data = pd.concat(df_list)
df = data.copy()
#change this as per the preprocesing used for training
#create all categorical variables that we did above for both training and test sets 
df['cabin_multiple'] = df.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
df['cabin_adv'] = df.Cabin.apply(lambda x: str(x)[0])
df['numeric_ticket'] = df.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
df['ticket_letters'] = df.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
df['name_title'] = df.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
#impute nulls for continuous data 
#df.Age = df.Age.fillna(training.Age.mean())
df.Age = df.Age.fillna(df.Age.median())
#df.Fare = df.Fare.fillna(training.Fare.mean())
df.Fare = df.Fare.fillna(df.Fare.median())
#drop null 'embarked' rows. Only 2 instances of this in training and 0 in test 
df.dropna(subset=['Embarked'],inplace = True)
#tried log norm of sibsp (not used)
df['norm_sibsp'] = np.log(df.SibSp+1)
# log norm of fare (used)
df['norm_fare'] = np.log(df.Fare+1)
# converted fare to category for pd.get_dummies()
df.Pclass = df.Pclass.astype(str)
#created dummy variables from categories (also can use OneHotEncoder)
all_dummies = pd.get_dummies(df[['PassengerId','Pclass','Sex','Age','SibSp','Parch','norm_fare','Embarked','cabin_adv','cabin_multiple','numeric_ticket','name_title']])
#scaling
encoded_df = all_dummies.copy()
encoded_df[['Age','SibSp','Parch','norm_fare']]= scale.fit_transform(encoded_df[['Age','SibSp','Parch','norm_fare']])
encoded_df['runtime']=runtime
encoded_df['modelname']='XGboost'
encoded_df['infertype']='Batch'
encoded_df['PassengerId']=df.PassengerId.apply(lambda x:str(x)+unqstr)
encoded_df.insert(0,'runtime',encoded_df.pop('runtime'))
encoded_df.insert(1,'modelname',encoded_df.pop('modelname'))
encoded_df.insert(2,'infertype',encoded_df.pop('infertype'))
encoded_df.insert(3,'PassengerId',encoded_df.pop('PassengerId'))
encoded_df.insert(4,'scorefilename',encoded_df.pop('scorefilename'))
#encoded_df.to_csv("score_prooutput.csv",index=False,header=False)
encoded_df.to_csv("/opt/ml/processing/output/test/"+filname+"_{}.csv".format(uuid.uuid1().time_low), index=False, header=False) # test data 

Overwriting titanic-score-preprocessing-xgb-script.py
